# Software design guidance, in Python

**[Arthur Goldberg](https://www.mountsinai.org/profiles/arthur-p-goldberg)**

This notebook was created for the [Biomedical Software Engineering](https://learn.mssm.edu/webapps/blackboard/content/listContentEditable.jsp?content_id=_448512_1&course_id=_5776_1 "Biomedical Software Engineering Blackboard site") course at the [Mount Sinai School of Medicine](https://icahn.mssm.edu/).


### Topics
+ Write small, reusable methods
+ 

This notebook contains examples of software problems and their solutions. The examples are taken from student programming assignments written in Python. They have been edited to best illustrate better solution methods.

## Write small, reusable methods
### Specified feature: ensure that all ids are unique
The program reads a set of records from a file, and must ensure that all ids in the records are unique. In particular, an errors must report any duplicated ids.
At this point in the code the records have been read and the code has ensured that each record has an id.

### Student approach
The student program does ensure that all ids are unique, but is overly complex and long because the duplicate detection is integrated into the data loading method.

In [3]:
import csv, sys

class Subject(object):
    def __init__(self, id, data):
        # error checking here removed from this example
        self.id = id
        self.data = data

    @classmethod
    def load_file(cls, file_name):
        """ Load subjects from a tab-separated value file into a list of Subject instances

        The file contains a header row. Each following row contains data about one subject.
        This method outputs error messages, including a list of duplicate ids.

        Args:
            file_name (:obj:'str'): path to a file of subjects

        Returns:
            (:obj:'list'): list of Subject instances formed from subject information in `file_name`
        """
        subjects = []
        with open(file_name) as csvfile:
            reader = csv.DictReader(csvfile, delimiter='\t')
            for row in reader:
                subject = cls(*row.values())
                subjects.append(subject)

        # detect duplicate subject ids
        subject_ids = [subject.id for subject in subjects]
        dup_id_row = []
        dup_ids = []
        for testid in set(subject_ids):
            if 1 < subject_ids.count(testid):
                for index, value in enumerate(subject_ids):
                    if value == testid:
                        dup_id_row.append(index + 2)  # + 2 because 1st row (header) are keys and index is zero-based
                        dup_ids.append(value)
        # duplicate detection done
        # dup_ids contains a list of duplicated ids, and dup_id_row contains their corresponding row numbers
        errors = []
        if dup_ids:
            for id, row in zip(dup_ids, dup_id_row):
                errors.append("{}: id {} duplicated ".format(row, id))
        if errors:
            sys.stderr.write('\n'.join(errors))
        return subjects

def save_test_data(file, data):
    with open(subjects_file, 'w') as file:
        for element in example_data:
            file.write('\t'.join(element) + '\n')

# create test data
example_data = [
    ['id', 'data'],
    ['id_3', 'data1'],
    ['id_4', 'data2'],
    ['id_3', 'data3'],
    ['id_6', 'data4'],
    ['id_6', 'data5'],
    ['id_3', 'data5']
]
subjects_file = 'subjects.tsv'
save_test_data(subjects_file, example_data)

Subject.load_file(subjects_file)

2: id id_3 duplicated 
4: id id_3 duplicated 
7: id id_3 duplicated 
5: id id_6 duplicated 
6: id id_6 duplicated 

### Problems with this approach
1. 8 lines of code perform duplicate detection, which is a specific problem that is distinct from reading in data and could be written in one, generic solution
2. If a generic method for duplicate detection were available, it could be used to detect and report duplicates in other software
3. Unnecessarily complex computationally: this approach takes $O(n^{2})$ time, which means that it grows at least as fast as the cube of the number of subjects

### Addressing these problems
+ Separate the issue of finding duplicates from the issues of reporting them as errors and of determining the rows in which they occur
+ Make a method that finds duplicates in a list
+ Make the method run fast, in $O(n)$ time
+ Use the method to find duplicates in the subjects
+ If it finds duplicates, use other data saved with the subjects to report the errors and the rows in which they occur

In [11]:
def find_dupes(ids):
    # return a set of the duplicates in ids; $O(n)$ complexity
    known_ids = set()
    duped_ids = set()
    for id in ids:
        if id in known_ids:
            duped_ids.add(id)
        known_ids.add(id)
    return duped_ids

# test find_dupes
assert find_dupes([1, 2, 1, 3]) == {1}
assert find_dupes([2, 1, 3]) == set()

class Subject(object):
    def __init__(self, id, data):
        # error checking here removed from this example
        self.id = id
        self.data = data

    @staticmethod
    def get_duped_subjects(subjects):
        # detect duplicate subject ids
        subject_ids = [subject.id for subject in subjects]
        duped_subject_ids = find_dupes(subject_ids)
        errors = []
        if duped_subject_ids:
            for id in duped_subject_ids:
                errors.append("id {} is duplicated ".format(id))
        return errors

    @classmethod
    def load_file(cls, file_name):
        """ Load subjects from a tab-separated value file into a list of Subject instances
        """
        subjects = []
        with open(file_name) as csvfile:
            reader = csv.DictReader(csvfile, delimiter='\t')
            for row in reader:
                subject = cls(*row.values())
                subjects.append(subject)
        errors = Subject.get_duped_subjects(subjects)
        if errors:
            sys.stderr.write('\n'.join(errors))
        return subjects

subjects = Subject.load_file(subjects_file)

id id_6 is duplicated 
id id_3 is duplicated 

### Benefits of this approach
1. We wrote a fast, simple, reusable generic method for duplicate detection. It takes $O(n)$ time, which is optimal.
2. It takes only 9 lines of code, and is tested a little.

### Problems with this approach
1. The row numbers of duplicated subject ids aren't reported.

### Addressing this problem
+ Save and use the row numbers of subjects

In [13]:
class Subject(object):

    def __init__(self, id, data, row_num): # CHANGED
        # error checking here removed from this example
        self.id = id
        self.data = data
        self._row_num = row_num # CHANGED

    @staticmethod
    def get_duped_subjects(subjects):
        # detect duplicate subject ids
        subject_ids = [subject.id for subject in subjects]
        duped_subject_ids = find_dupes(subject_ids)
        errors = []
        if duped_subject_ids:
             # START CHANGED
            for subject in subjects:
                if subject.id in duped_subject_ids:
                    errors.append("{}: id {} is duplicated ".format(subject._row_num, subject.id))
             # END CHANGED
        return errors

    @classmethod
    def load_file(cls, file_name):
        """ Load subjects from a tab-separated value file into a list of Subject instances
        """
        subjects = []
        row_num = 2 # CHANGED
        with open(file_name) as csvfile:
            reader = csv.DictReader(csvfile, delimiter='\t')
            for row in reader:
                subject = cls(*row.values(), row_num) # CHANGED
                subjects.append(subject)
                row_num += 1 # CHANGED
        errors = Subject.get_duped_subjects(subjects)
        if errors:
            sys.stderr.write('\n'.join(errors))
        return subjects

subjects = Subject.load_file(subjects_file)

2: id id_3 is duplicated 
4: id id_3 is duplicated 
5: id id_6 is duplicated 
6: id id_6 is duplicated 
7: id id_3 is duplicated 

### Final remarks
1. Duplicated ids are reported in row order
2. We have a reusable duplicate detection method
3. We should think about where this method belongs
4. Subjects store their row numbers, which may be handy for other purposes